# Import eines Pickle-Files

In [76]:
import pandas as pd
import pickle
import os

rootdir = os.getcwd() + "/data/training/"

print(rootdir)

data = []
labels = []

for subdir, dirs, files in os.walk(rootdir):
    for fl in files:
        if fl.endswith('.pkl'):
            # string split for event labelling
            str_arr = fl.split('_')
            label   = str_arr[1]

            # read data and append it to variable
            infile = open((rootdir + fl),'rb')
            new_dict = pickle.load(infile, encoding='latin1')

            data.append(new_dict)
            labels.append(label)

print(data[0].keys())


/Users/maximilianrohleder-kirsch/angewandte-ki-cog-village/data/training/
dict_keys(['Magnetometer', 'JinsBlinkSpeed', 'rightHand', 'Accelerometer', 'LinearAccelerometer', 'JinsGyroscope', 'JinsEyeMovement', 'Gravity', 'label', 'MSAccelerometer', 'executionId', 'session', 'JinsAccelerometer', 'JinsBlinkStrength', 'MSGyroscope', 'Gyroscope', 'subject'])


In [142]:
import numpy as np

arr = []
keys = [
    'Magnetometer', 
    'Accelerometer', 
    'LinearAccelerometer', 
    'JinsGyroscope', 
    'Gravity', 
    'MSAccelerometer',
    'JinsAccelerometer',
    'MSGyroscope',
    'Gyroscope',
    'label'
]

ary = []
for dt in data:
    internal = []
    for key in keys[:-1]:
        feature = dt[key]
        for column in feature.T:
            internal.append(np.mean(column))
            internal.append(np.var(column))
    internal.append(dt["label"])
    ary.append(internal)

In [144]:
df = pd.DataFrame(ary)
df

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,-23.966837,0.187867,-17.618367,1.393291,29.771173,0.355786,2.983384,0.003677,2.658176,0.001944,...,0.476318,-0.722043,0.489340,0.000584,0.000035,0.003134,0.000055,-0.003458,0.000025,2
1,2.798780,129.802689,-8.214177,19.302309,38.190041,3.848031,-1.847596,1.258675,0.591277,1.443341,...,533.439191,2.169103,1553.827547,-0.014283,0.009285,-0.063902,0.007765,-0.264008,0.139377,3
2,-6.954337,2.436643,10.458929,1.802538,40.122194,0.395958,-0.169665,0.458301,2.761107,0.109442,...,180.411374,-2.763205,114.037781,-0.007285,0.002771,-0.006399,0.003510,0.128388,0.115336,3
3,16.001008,0.384670,-42.006048,0.828025,-1.447833,0.591107,-0.751700,0.011161,10.340806,0.003299,...,41.975151,-2.147697,54.859132,0.004364,0.000551,-0.001383,0.002474,0.001153,0.000310,4
4,30.602296,0.206197,-15.069388,1.097960,12.032908,1.367811,-9.117331,0.015469,1.656084,0.016881,...,234.814662,-24.871052,4104.794009,0.001805,0.000744,0.009822,0.001750,-0.004913,0.000481,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,-21.329268,376.385834,-13.782520,38.920881,-17.001016,458.573519,4.013327,29.094279,1.411953,0.879282,...,1084.992349,-16.703941,1462.954276,0.093955,0.116498,-0.295426,0.425693,0.092752,0.128918,1
70,-3.845395,0.244458,-7.397773,0.752233,38.065283,0.347016,2.520694,0.003014,-0.201926,0.001981,...,23.492894,2.517137,193.491400,0.001462,0.000032,0.003619,0.000024,-0.002479,0.000024,1
71,15.155742,0.858986,-37.024136,0.992256,13.865854,4.165376,-0.565536,0.095867,10.728565,0.176493,...,2884.469233,6.376335,10598.898821,0.008932,0.007115,0.016136,0.017093,0.000290,0.001966,4
72,10.003049,69.599108,-35.783283,23.300833,21.618648,71.522206,-0.108634,2.427299,10.084150,5.118462,...,1017.192958,2.908872,9089.813250,0.038496,1.098707,0.175277,1.479192,0.065239,0.146996,5


# Alternative Way of concetanating the data

In [14]:
import numpy as np 
from collections import defaultdict

ACTIVITIES = { 
    "Bending" : 0,
    "Lying": 1,
    "Sitting": 2,
    "Squatting": 3,
    "Standing": 4,
    "Walking": 5
}

keys = list(data[0].keys())
unique_labels = list(set(labels))

activity_multi_arr = []
activity_labels = unique_labels

for label in unique_labels:
    data_total = defaultdict(list)

    for dt in data:
        keys = list(dt.keys())
        if dt["label"] == ACTIVITIES[label]:
            for k in keys:
                data_total[k].append(dt[k])

    for k in keys:
        data_total[k] = np.vstack(data_total[k])

    activity_multi_arr.append(data_total)

In [12]:
arr_magneto = []
arr_jins_blinkspeed = []

for row in df.iloc[:, 0]:
    arr_magneto.append(pd.DataFrame(row.reshape(-1, row.shape[1])))

df1 = pd.concat(arr_magneto)

for row in df.iloc[:, 1]:
    arr_jins_blinkspeed.append(pd.DataFrame(row.reshape(-1, row.shape[1])))

df1


,0,1,2
0,-9.125,-11.6875,40.8750
1,-8.375,-9.8125,39.6250
2,-8.375,-11.6875,40.4375
3,-9.125,-11.6875,38.8125
4,-9.125,-9.8125,39.6250
...,...,...,...
3183,-15.625,-41.5000,20.4375
3184,-14.875,-39.6250,21.3125
3185,-15.625,-40.0000,23.3750
3186,-15.250,-38.5625,24.2500
